# Goal 

To forecast bike rental demand in the Capital Bikeshare program in Washington, D.C. by combining historical usage patterns with weather data in order to forecast bike rental demand 

# Insights on bike sharing 

The first bike sharing projects were initiated by local community organisations, or as charitable projects intended for the disadvantaged, or to promote bicycles as a non-polluting form of transport, or they were business enterprises to rent out bicycles. Ernest Callenbach 's novel Ecotopia (1975) illustrated the idea. 

# Wiki 

A bicycle-sharing system, public bicycle scheme,or public bike share (PBS) scheme,is a service in which bicycles are made available for shared use to individuals on a short term basis for a price or free. Many bike share systems allow people to borrow a bike from a "dock" and return it at another dock belonging to the same system. Docks are special bike racks that lock the bike, and only release it by computer control. The user enters payment information, and the computer unlocks a bike. The user returns the bike by placing it in the dock, which locks it in place. Other systems are dockless.


# Advantages of bike-sharing systems 

They use smart-cards to allow the bicycles to be returned to any station in the system, which facilitates one-way rides to work, education or shopping centres. Thus, one bike may take 10–15 rides a day with different users and can be ridden up to 10,000 km

The data generated by these systems makes them attractive for researchers because the duration of travel, departure location, arrival location, and time elapsed is explicitly recorded. Bike sharing systems therefore function as a sensor network, which can be used for studying mobility in a city.

# For many systems, smartphone mapping apps show nearby available bikes and open docks. In July 2020, Google Maps began including bike shares in its route recommendations



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import pandas as pd

In [ ]:
filename = ("../input/bike-sharing-demand/train.csv")

train=pd.read_csv(filename)

train.head()

# Attribute Information:

datetime - hourly date + timestamp
season - 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather - 
* 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
* 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
* 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
* 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals

# My Hypothesis 

General factors that would impact the prediction of increase in bike demand for rentals are -

1) Favourable seasons to enjoy bike ride 

2) Clear Weather conditions suitable for bike riding

3) Holidays for biking recreation purposes

4) Working days - Fitness enthusiasts who may commute to schools/colleges/work during weekdays

In [ ]:
train.shape

In [ ]:
train.isnull().sum() 


In [ ]:
train.describe()

In [ ]:
train.info()

There are No null values.


There are some Columns which have interger values for season, holiday,weather,working day columns but these should be categorical data types. We need to encode these so that the system understands the values.


The datetime column is an object with format YYYY Year MM Month and DD Date , appears as a string and we need to convert to "datetime" datatype so that we can leverage the date time library and utiliize that method for further helpful analysis

In [ ]:
train.datetime = pd.to_datetime(train.datetime)
train.datetime = pd.to_datetime(train.datetime)

train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour

With the above code the datetime has been changed from string to a new data type called datetime as required

In [ ]:
train.info()

In [ ]:
train.head()

# Visualizing correlation between Counts and Features

In [ ]:
corr_matrix = train.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr_matrix, annot=True)
plt.show()

# Correlation analysis from above matrix  

Lets understand the relationship between cnt (i.e bike rental count) and all other features.

Positively correlated featurea are - temperature,season

Negative correlated features are below  -


Humidity - as humidity goes higher the bike count reduces as its not a favourable condition for biking.


Temp and Atemp - we can see they are highly corelated to each other so we can research to find out if they are redundant features


Weather 

# Further preprocessing steps :

Dummy variables Creation

In the dataset we have some categorical variables like season, weather, holiday,working day


Inorder to include these in our model, we'll need to make dummy variables so that the system understands these values in separate columns . Hence we will use pandas get_dummies() as below - 

In [ ]:
dummy_fields = ['season', 'weather', 'holiday', 'workingday']
for each in dummy_fields:
    dummies = pd.get_dummies(train[each], prefix=each, drop_first=False)
    train = pd.concat([train, dummies], axis=1)

fields_to_drop = ['datetime','season', 'weather','holiday','atemp','workingday']
data = train.drop(fields_to_drop, axis=1)


In [ ]:
data.head()

# Data Visualization 

From the below plots we can analyze as follows - 

# 1st box plot 
count shows there are some outliers beyond 600 too

# 2nd plot 
count against season shows (1:spring has least rentals, followed by 4:winter, season 2 & 3 :during summer and fall seasons the bike rentals count is highest so these bikers prefer warmer climates that cold climate.

# 3rd plot 
count against months shows that from March the bike rentals are progressively increasing till July.
For the Year beginning and ending months the bike rentals are least due to the holiday season and cold climate. The variations pattern observed with months is similar to seasons pattern plot.

# 4th plot 
count against month and working day- it is observed that for working days the rental count is higher than non working day which is indicative that bikes are being rented for commuting to offices/schools



In [ ]:
fig,axes = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(16,14)
sns.boxplot(data=train, y='count', orient='v', ax=axes[0][0])
sns.boxplot(data=train, y='count', x='season',orient='v', ax=axes[0][1])
sns.boxplot(data=train, y='month', x='count',orient='h', ax=axes[1][0])
sns.boxplot(data=train, y='month', x='count', orient='h', hue='workingday',ax=axes[1][1])

From the below plot 'Variation of count with hours' we see that the count of bike rental during morning hours (7am - 9 am) and in the evening (5pm - 7 pm) is significantly high when compared to the other time intervals. This evident pattern is because they are using it for commuting to and from office/schools

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.boxplot(data=data,x="hour", y="count")
ax.set_title('Variation of count with hours')
sns.set(style="whitegrid")

From the below graph 'Variation of hourly count on season for Registered Users_hours' we see that Hourly count of bike rentals for registered users shows that the demand is high during peak morning hours (7 am - 9 am) and evening hours(5 pm - 7pm) which re-emphasizes bikes are being used for commuting to office/school

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.pointplot(x='hour', y='registered', data=train, hue='season')
ax.set_title('Variation of hourly count on season for Registered Users_hours')
sns.set(style="whitegrid")

From the below graph plotted for 'Variation of bike rental count with windspeed' we can see people prefer moderate windspeed as bike rental counts are highest for low windspeed.

In [ ]:
windspeed_reg = data.groupby(['windspeed'])['registered'].mean()
windspeed_cas = data.groupby(['windspeed'])['casual'].mean()
fig,axes = plt.subplots(figsize=(12, 6))

windspeed_reg.plot(kind = 'line', marker='X', markerfacecolor='maroon', markersize= 10, color='coral', linewidth=3)
windspeed_cas.plot(kind = 'line', marker='X', markerfacecolor='purple', markersize= 10, color='deeppink', linewidth=3)
print('Variation of bike rental count with windspeed')

plt.legend(loc='upper left')
plt.xlabel('Windspeed')
plt.ylabel('Rental count')
plt.show()

Lets spearhead our analysis and look at the renting behaviour based on Season 

In [ ]:
df_season_spring = train[train['season'] ==1]
df_season_spring['month'].unique()

In [ ]:
df_season_summer = train[train['season'] ==2]
df_season_summer['month'].unique()

In [ ]:
df_season_fall = train[train['season'] ==3]
df_season_fall['month'].unique()

In [ ]:
df_season_winter = train[train['season'] ==4]
df_season_winter['month'].unique()

In [ ]:
df_season_summer.month.nunique()

Lets analyse to see Which month of each season has highest rent and how is weather predicting on rents

Spring and Weather situation


weathersit reference:

1: Clear, Few clouds, Partly cloudy, Partly cloudy

2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

Very Sparse Weather condition of 4: Heavy Rain situation in these months

When weather situation is clear rentals are more

In [ ]:
sns.factorplot('month','count',hue='weather',data=df_season_spring,ci=None)
plt.show()

Summer and weather situation


weathersit :

1: Clear, Few clouds, Partly cloudy, Partly cloudy

2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

There is NO 4: Heavy Rain situation in these months

The rentals are linearly increasing and high in count

In [ ]:
sns.factorplot('month','count',hue='weather',data=df_season_summer,ci=None)
plt.show()

Fall and weather situation

weathersit :

1: Clear, Few clouds, Partly cloudy, Partly cloudy

2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

There is NO 4: Heavy Rain situation in these months

When the weather is clear the rentals are high in count and as the weather shows signs of few clouds or light snow the rentals drop in count.



In [ ]:
sns.factorplot('month','count',hue='weather',data=df_season_fall,ci=None)
plt.show()

Winter and weather situation

The bike rentals drop due to the cold winters

In [ ]:
sns.factorplot('month','count',hue='weather',data=df_season_winter,ci=None)
plt.show()

Let's analyse Demand of bikes across the month.

You will see from below graph Highest demand of bikes are in June , July, Aug, Sept, Oct

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.barplot(x='month', y='count', data=data)
ax.set_title('Variation of count with months')
sns.set(style="whitegrid")

tick_val=[0,1 , 2, 3, 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11]
tick_lab=['Jan','Feb','March','April','May','June','July','Aug','Sept','Oct','Nov','Dec']
plt.xticks(tick_val, tick_lab)
plt.show()

Its important to scale our data now 

Scaling target variables To make training the network easier, we'll standardize each of the continuous variables by scaling the variables such that they have zero mean and a standard deviation of 1.

SD=(x−x¯)μ

In [ ]:
quant_features = ['temp', 'humidity', 'windspeed']

scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

data.head()

In [ ]:
data.shape

In [ ]:
# Separate the data into features and targets
target_fields = ['count', 'casual', 'registered']

X = data.drop(target_fields, axis=1)
y = data['count']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
print("Train_set: ", X_train.shape)
print("Test_set: ", X_test.shape)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=500, max_features=10,min_samples_leaf=3,random_state=43)
rf.fit(X_train, y_train)

rf_prediction = rf.predict(X_test)
from sklearn.metrics import mean_squared_error
from sklearn import metrics
print('MSE:', metrics.mean_squared_error(y_test, rf_prediction))

In [ ]:
print(rf.score(X_train,y_train))
print(rf.score(X_test,y_test))

In [ ]:
X_train.shape
y_train.shape

In [ ]:
X_test.shape


In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(min_samples_split=25,random_state=0)

dt_reg.fit(X_train, y_train)
dt_prediction = dt_reg.predict(X_test)
print('MSE:', metrics.mean_squared_error(y_test, dt_prediction))

In [ ]:
test=pd.read_csv('../input/bike-sharing-demand/test.csv')

test.datetime = pd.to_datetime(train.datetime)

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour

In [ ]:
datetime_col=test.datetime

In [ ]:
test.info()

In [ ]:
test.shape

In [ ]:
dummy_fields = ['season', 'weather', 'holiday', 'workingday']
for each in dummy_fields:
    dummies = pd.get_dummies(test[each], prefix=each, drop_first=False)
    test = pd.concat([test, dummies], axis=1)

fields_to_drop = ['datetime','season', 'weather','holiday','atemp','workingday']
data1 = test.drop(fields_to_drop, axis=1)

In [ ]:
test_season_spring = test[test['season'] ==1]
test_season_spring['month'].unique()

In [ ]:
test_season_summer = test[test['season'] ==2]
test_season_summer['month'].unique()

In [ ]:
test_season_fall = test[test['season'] ==3]
test_season_fall['month'].unique()

In [ ]:
test_season_winter = test[test['season'] ==4]
test_season_winter['month'].unique()

In [ ]:
quant_features = ['temp', 'humidity', 'windspeed']

scaled_features = {}
for each in quant_features:
    mean, std = data1[each].mean(), data1[each].std()
    scaled_features[each] = [mean, std]
    data1.loc[:, each] = (data1[each] - mean)/std

data1.head()

In [ ]:
data1.shape

In [ ]:
X_test=data1.iloc[:,:].values
X_test.shape

In [ ]:
y_output=rf.predict(X_test)
y_output

In [ ]:
rounded_final=list(map(int,map(round,y_output)))

In [ ]:
len(y_output)

In [ ]:
X_test

In [ ]:
len(datetime_col)

In [ ]:
import pandas as pd 
samplesubmission = pd.read_csv("../input/bike-sharing-demand/sampleSubmission.csv")
samplesubmission["count"]=rounded_final

In [ ]:
samplesubmission.head()

In [ ]:
samplesubmission.to_csv("final_submission.csv",index=False)

In [ ]:
samplesubmission.info()

# Motivation : "Life is a continuous exercise in creative problem solving" 